# NLP with Bert for Sentiment Analysis

### Importing the libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
path =  "drive/My Drive/nlp bert"

In [3]:
import os

os.chdir(path)

In [4]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 125kB/s 
     |████████████████████████████████| 421.8MB 16kB/s 
     |████████████████████████████████| 6.7MB 36.4MB/s 
     |████████████████████████████████| 983kB 14.4MB/s 
     |████████████████████████████████| 245kB 34.7MB/s 
     |████████████████████████████████| 778kB 31.9MB/s 
     |████████████████████████████████| 471kB 40.7MB/s 
     |████████████████████████████████| 3.9MB 40.2MB/s 
     |████████████████████████████████| 450kB 38.9MB/s 
     |████████████████████████████████| 3.0MB 38.1MB/s 
     |████████████████████████████████| 890kB 38.2MB/s 
     |████████████████████████████████| 1.1MB 36.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.18.3-cp36-none-any.whl size=25252749 sha256=24eba4b9e4900b79cb9d2047f17add50c7818dbddee5fc34fb932f0fd0896a98
  Stored in directory: /root/.cache/pip/wheels/21/e3/5b/fb6dc301116865d1a4a0732a7fb91bdbfec1e936c60536826d
  Created wheel for keras-bert: filename=keras_bert-0.85.0-cp36-no

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
import ktrain
from ktrain import text

## Part 1: Data Preprocessing

In [167]:
train = pd.read_csv('train_2kmZucJ.csv')
test = pd.read_csv('test_oJQbWVk.csv')

In [7]:
train.shape

(7920, 3)

In [168]:
test.shape

(1953, 2)

In [9]:
train.head()

,id,label,tweet
0,1,0,#fingerprint #Pregnancy Test https://goo.gl/h1...
1,2,0,Finally a transparant silicon case ^^ Thanks t...
2,3,0,We love this! Would you go? #talk #makememorie...
3,4,0,I'm wired I know I'm George I was made that wa...
4,5,1,What amazing service! Apple won't even talk to...


In [169]:
test.head()

,id,tweet
0,7921,I hate the new #iphone upgrade. Won't let me d...
1,7922,currently shitting my fucking pants. #apple #i...
2,7923,"I'd like to puts some CD-ROMS on my iPad, is t..."
3,7924,My ipod is officially dead. I lost all my pict...
4,7925,Been fighting iTunes all night! I only want th...


### Creating the training and test sets

In [22]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(train,
                                                                   text_column='tweet',
                                                                   label_columns=['label'],
                                                                   maxlen=500,
                                                                   preprocess_mode='bert')

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


## Part 2: Building the BERT model

In [23]:
model = text.text_classifier(name='bert',
                             train_data=(x_train, y_train),
                             preproc=preproc)

Is Multi-Label? False
maxlen is 500
done.


## Part 3: Training the BERT model

In [24]:
learner = ktrain.get_learner(model=model,
                             train_data=(x_train, y_train),
                             val_data=(x_test, y_test),
                             batch_size=6)

In [25]:
learner.fit_onecycle(lr=2e-5,
                     epochs=4)



begin training using onecycle policy with max lr of 2e-05...
Train on 7128 samples, validate on 792 samples
Epoch 1/4
7128/7128 [==============================] - 2077s 291ms/sample - loss: 0.2743 - accuracy: 0.8784 - val_loss: 0.1962 - val_accuracy: 0.9167
Epoch 2/4
7128/7128 [==============================] - 2064s 290ms/sample - loss: 0.1951 - accuracy: 0.9223 - val_loss: 0.2100 - val_accuracy: 0.9129
Epoch 3/4
7128/7128 [==============================] - 2068s 290ms/sample - loss: 0.1357 - accuracy: 0.9533 - val_loss: 0.2606 - val_accuracy: 0.8990
Epoch 4/4
7128/7128 [==============================] - 2064s 290ms/sample - loss: 0.0654 - accuracy: 0.9806 - val_loss: 0.2805 - val_accuracy: 0.9167


In [26]:
learner.validate(val_data=(x_test, y_test))

              precision    recall  f1-score   support

           0       0.96      0.93      0.94       596
           1       0.81      0.87      0.84       196

    accuracy                           0.92       792
   macro avg       0.88      0.90      0.89       792
weighted avg       0.92      0.92      0.92       792



array([[556,  40],
       [ 26, 170]])

## How to Use Our Trained BERT Model

We can call the `learner.get_predictor` method to obtain a Predictor object capable of making predictions on new raw data.

In [27]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [28]:
predictor.get_classes()

['0', '1']

In [170]:
test_tweets = test['tweet'].values
test_tweets

array(["I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks",
       'currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://instagr.am/p/UUIS0bIBZo/',
       "I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the screen?\n",
       ...,
       'There goes a crack right across the screen. If you could actually provide a more durable screen, that would be great #Apple #Iphone',
       '@codeofinterest as i said #Adobe big time we may well as include #apple to',
       'Finally I got it .. thanx my father .. #Samsung #galaxy #s3 #gift #father #phone #new http://instagr.am/p/NoxkiPE'],
      dtype=object)

In [171]:
test_tweets[0]

"I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks"

In [172]:
predictor.predict(test_tweets[0])

'1'

In [173]:
test_tweets[1]

'currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://instagr.am/p/UUIS0bIBZo/'

In [174]:
predictor.predict(test_tweets[1])

'1'

In [175]:
test_tweets[179]

'Welcome #samsung #galaxy #note3 #family #group #telephone #gmail #selfie ;)) http://instagram.com/p/mfwV3ZhzVq/'

In [176]:
predictor.predict(test_tweets[179])

'0'

In [177]:
test_pred = predictor.predict(test_tweets)

In [178]:
test_pred

['1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0'

In [179]:
test['label'] = test_pred

submission = test[['id','label']]

submission.to_csv('result.csv', index=False)

In [180]:
res = pd.read_csv('result.csv')

In [181]:
res

,id,label
0,7921,1
1,7922,1
2,7923,1
3,7924,1
4,7925,1
...,...,...
1948,9869,1
1949,9870,0
1950,9871,1
1951,9872,1


The `predictor.save` and `ktrain.load_predictor` methods can be used to save the Predictor object to disk and reload it at a later time to make predictions on new data.